In [3]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import xarray as xr
import re
import rtree
import shapely
import pickle

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

## File Paths

In [4]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset

tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
tiles_xml_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_complete_dataset_path, exist_ok=True)

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)

In [ ]:
result= cv2.matchTemplate(gray_tile, gray_image, cv2.TM_CCOEFF)
min_val, max_val, min_loc, max_loc= cv2.minMaxLoc(result)

height, width= gray_image.shape[:2]

top_left= max_loc
bottom_right= (top_left[0] + width, top_left[1] + height)
#cv2.rectangle(tile, top_left, bottom_right, (0,0,255),5)

cv2.imshow('Rainforest', tile)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
#len(
remaining_standard_names = []
for formatted_name in incorrect_labeled_chip_names_by_subfolder.loc[gray_incorrect_labeled_chip_image_unique_indices]["incorrect_chip_names"]:
    standard_name = formatted_name.split("_",4)[4] 
    remaining_standard_names.append(standard_name.rsplit("_",1)[0])

remaining_standard_names = np.unique(remaining_standard_names)
remaining_standard_names

array(['m_2608063_ne_17_060_20191120', 'm_2608064_nw_17_060_20191120',
       'm_2708205_ne_17_060_20191129', 'm_2708205_nw_17_060_20191129',
       'm_2708205_se_17_060_20191129', 'm_2708205_sw_17_060_20191129',
       'm_2708212_ne_17_060_20191129', 'm_2908901_ne_16_060_20190707',
       'm_2908901_se_16_060_20190707', 'm_2908901_sw_16_060_20190707',
       'm_2908909_nw_16_060_20190707', 'm_2908917_nw_16_060_20190707',
       'm_2908917_sw_16_060_20190707', 'm_2909002_nw_15_060_20190707',
       'm_2909006_nw_15_060_20190707', 'm_2909006_sw_15_060_20190707',
       'm_2909007_se_15_060_20190707', 'm_2909007_sw_15_060_20190707',
       'm_2909008_se_15_060_20190707', 'm_2909008_sw_15_060_20190707',
       'm_2909016_ne_15_060_20190707', 'm_2909016_se_15_060_20190707',
       'm_2909017_sw_15_060_20190707', 'm_2909019_sw_15_060_20190707',
       'm_2909029_se_15_060_20190707', 'm_2909038_ne_15_060_20190707',
       'm_2909038_se_15_060_20190707', 'm_2909055_sw_15_060_20190830',
      

In [46]:
for directory in tqdm.tqdm(unverified_verified_set1_subfolders_paths):
    chips_negative = glob(os.path.join(os.path.dirname(directory[0]),'chips_negative')+"/*.jpg")
    for os.stat(os.path.join(dups_chips_positive_path,image)).st_size

100%|██████████| 44/44 [02:57<00:00,  4.03s/it]


In [ ]:
gray_incorrect_labeled_chip_image_array[temp_gray_incorrect_labeled_chip_image_unique_indices], gray_correct_image

In [19]:
np.save("temp_gray_incorrect_labeled_chip_image_unique_indices.npy", temp_gray_incorrect_labeled_chip_image_unique_indices)

In [58]:
os.stat(incorrect_labeled_chip_names_by_subfolder.loc[incorrect_labeled_unique_index]["incorrect_chip_paths"])

os.stat_result(st_mode=33206, st_ino=30519315530, st_dev=10131971, st_nlink=1, st_uid=0, st_gid=0, st_size=40897, st_atime=1645729341, st_mtime=1614550504, st_ctime=1619458129)

In [ ]:
gray_incorrect_labeled_chip_image_unique_indices

In [47]:
"""
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.set_title('correct_image')
ax1.imshow(correct_image)
ax2.set_title('labeled_chip_array')
ax2.imshow(labeled_chip_array)
plt.show() 
"""

In [33]:
###tiles that are likely contain the correct source data for duplicate images
fc.directory_tile_names("C:/chip_allocation/complete_dataset/dups_chips_positive",
                        'dup tile names')

dup_tile_names = pd.read_csv('dup tile names.csv')
standard_tile_names = [] 
for formatted_tile_names in dup_tile_names['dup tile names']:
    standard_tile_names.append(formatted_tile_names.split("_",4)[4])
dup_unique_standard_tile_names = np.unique(standard_tile_names)

In [34]:
dup_unique_standard_tile_names

array(['m_2608064_nw_17_060_20191120', 'm_2708205_nw_17_060_20191129',
       'm_2708205_se_17_060_20191129', 'm_2708205_sw_17_060_20191129',
       'm_2708212_ne_17_060_20191129', 'm_2908901_ne_16_060_20190707',
       'm_2908901_se_16_060_20190707', 'm_2908901_sw_16_060_20190707',
       'm_2908917_nw_16_060_20190707', 'm_2909002_nw_15_060_20190707',
       'm_2909006_nw_15_060_20190707', 'm_2909006_sw_15_060_20190707',
       'm_2909007_se_15_060_20190707', 'm_2909007_sw_15_060_20190707',
       'm_2909008_se_15_060_20190707', 'm_2909008_sw_15_060_20190707',
       'm_2909063_nw_15_060_20190830', 'm_2909309_se_15_060_20190726',
       'm_2909310_sw_15_060_20190726', 'm_2909635_ne_14_060_20181201',
       'm_3008135_se_17_060_20191119', 'm_3008136_se_17_060_20191119',
       'm_3008136_sw_17_060_20191119', 'm_3008143_ne_17_060_20191119',
       'm_3008144_nw_17_060_20191119', 'm_3008824_ne_16_060_20191115',
       'm_3008844_ne_16_060_20181204', 'm_3008845_nw_16_060_20181204',
      

In [70]:
#correct_images_from_chipped_tile_for_positive_images(tiles_complete_dataset_path, incorrect_labeled_chip_names_by_subfolder, incorrect_labeled_chip_image_array)
correct_images_from_chipped_tile_for_positive_images(tiles_complete_dataset_path, dup_unique_standard_tile_names, gray_incorrect_labeled_chip_image_unique_array)

  0%|          | 0/87 [00:00<?, ?it/s]

//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\tiles\m_2608064_nw_17_060_20191120.tif


  0%|          | 0/87 [19:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
index = np.where(unverified_verified_set1_image_paths == unverified_verified_set1_image_paths[0])
print(index)
#labeled_chip_array = cv2.imread(os.path.join(images_path[index[0]])) #open image
#first folder in unverified
#get directories that need to be check
#positive_images = np.zeros((0, 512, 512, 3))
#identify tiles that have corresponding images in directory

## File Paths

ValueError: list.remove(x): x not in list

In [ ]:
###CONFIRM CHIP X AND Y INDEXES

In [ ]:
# Deal with incorrectly labeled images 

### Identifical duplicate tiles 

In [ ]:
complete_dataset_chips_positive_path = os.path.join(parent_directory,"complete_dataset/chips_positive")
complete_dataset_chips_positive = os.listdir(complete_dataset_chips_positive_path)
#for chip in complete_dataset_chips_positive:
#complete_dataset_chips_positive[0].split[_]
fc.remove_thumbs(complete_dataset_chips_positive_path)
formatted_tiles = []
for path in complete_dataset_chips_positive:
    img = os.path.splitext(path)[0] #name of tif with the extension removed
    tile = img.rsplit("_",1)[0]
    #print(tile)
    formatted_tiles.append(tile)
    
formatted_tiles = np.unique(formatted_tiles)
len(formatted_tiles)


standard_tiles = []
for tile in formatted_tiles:
    standard_tiles.append(tile.split("_",4)[4]) #get the tile names to remove duplicates from being downloaded
    
u, c = np.unique(standard_tiles, return_counts = True)
duplicate_tiles = u[c > 1]
len(duplicate_tiles)
duplicate_tiles

In [ ]:
#index over images that do not match 
index, = np.where(correct_0_incorrect_1_images == 1)
image_in_directory_array = images_in_directory_array[index[0]] #use the actual value of index (saved as an array)

In [ ]:
### Deal with duplicates

There are possibly duplicate images that are misnamed from when Qianyu was in charge

In [ ]:
fc.remove_thumbs("C:/chip_allocation/complete_dataset/chips_positive")

In [ ]:
state = []
resolution = []
year = []
capture_date  = []
utm_zone  = []

standard_tile_names = []
chip_names = []
NW_coordinates = []
SE_coordinates = []
row_indicies = []
col_indicies = []
full_path  = []
root = []
for tile_name in tqdm.tqdm(['m_4009152_se_15_060_20190728.tif']): #index over the tiles in the tiles_dir 
    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)
    count = 1      

    item_dim = int(512)          
    tile = cv2.imread(os.path.join(tiles_dir, tile_name)) 
    tile_height,  tile_width,  tile_channels = tile.shape #the size of the tile 
    print(tile_height,tile_width)

    #divide the tile into 512 by 512 chips (rounding up)
    row_index = math.ceil(tile_height/512) 
    col_index = math.ceil(tile_width/512)
    print(row_index*512,col_index*512)

    for x in range(0, col_index):
        for y in range(0, row_index):
            #Tile names no longer match chip file names!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            chip_name_temp = file_name+ '_'+ str(count).zfill(6) + '.jpg'
            count += 1  
            if len(verified_positive_jpgs[verified_positive_jpgs[:,0] == chip_name_temp]) > 0: #only record values for images that are annotated
                #image characteristics
                chip_names.append(chip_name_temp) # The index is a six-digit number like '000023'.
                NW_coordinates.append([x*item_dim, y*(item_dim)]) #NW (Top Left) 
                SE_coordinates.append([x*item_dim+item_dim-1, y*(item_dim)+item_dim-1]) #SE (Bottom right) 
                row_indicies.append(y)
                col_indicies.append(x)
                #tile characteristics
                ## Get tile url using tile name
                standard_tile_names.append(tile_name)
                #path
                full_path = verified_positive_jpgs[verified_positive_jpgs[:,0] == chip_name_temp][0][1]
                root = full_path.split("\\",2)[0]

In [ ]:
chip_names
m_4009152_se_15_060_20190728

In [ ]:
tile_names_tile_urls_complete_array_unique_standard_tile_names[tile_names_tile_urls_complete_array_unique_standard_tile_names[:,0]=="m_3812263_sw_10_060_20180723_20190210.tif"]

In [ ]:
tile_name_tile_url = np.load('image_download_azure/tile_name_tile_url_complete_array.npy')
tile_name_tile_url[tile_name_tile_url[:,0]=='m_3812263_sw_10_060_20180723_20190210.tif']

In [ ]:
ap.download_url(tile_names_tile_urls_complete_array_unique_standard_tile_names[tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]=="m_3812263_sw_10_060_20180723"][0][1],
                "C:/chip_allocation/test/tiles",
                destination_filename = tile_names_tile_urls_complete_array_unique_standard_tile_names[tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]=="m_3812263_sw_10_060_20180723"][0][0],       
                progress_updater=ap.DownloadProgressBar())

In [ ]:
for tile_name in os.listdir("C:/chip_allocation/test/tiles"): #index over the tiles in the tiles_dir 
    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)
    item_dim = int(512)
    count = 1            
    tile = cv2.imread(os.path.join("C:/chip_allocation/test/tiles", tile_name)) 
    tile_height,  tile_width,  tile_channels = tile.shape #the size of the tile 

    #divide the tile into 512 by 512 chips (rounding up)
    row_index = math.ceil(tile_height/512) 
    col_index = math.ceil(tile_width/512)
    #print(row_index, col_index)

    for y in range(0, col_index):
        for x in range(0, row_index):
            chip_img = tile[y*item_dim:y*item_dim+item_dim, x*(item_dim):x*(item_dim)+item_dim]

            #specify the path to save the image
            chips_save_path = os.path.join("C:/chip_allocation/test/chips", file_name+ '_'+ \
                       str(count).zfill(6) + '.jpg') # The index is a six-digit number like '000023'.

            #add in back space if it is the edge of an image
            if chip_img.shape[0] != 512:  #Height
                #print("Incorrect Height")
                black_height = 512  - chip_img.shape[0] #Height
                black_width = 512 #- chip_img.shape[1] #width
                black_img = np.zeros((black_height,black_width,3), np.uint8)
                #print(black_img.shape[0]) #Height
                #print(black_img.shape[1]) #width
                chip_img = np.concatenate([chip_img, black_img])

            if chip_img.shape[1] != 512: #width
                #print("Incorrect Width")
                black_height = 512 #- chip_img.shape[0] #Height
                black_width = 512 - chip_img.shape[1] #width
                black_img = np.zeros((black_height,black_width,3), np.uint8)
                #print(black_img.shape[0]) #Height
                #print(black_img.shape[1]) #width
                chip_img = np.concatenate([chip_img, black_img],1)

            #save image
            cv2.imwrite(os.path.join(chips_save_path), chip_img)    
            #counter for image pathway
            count += 1  
    print(count)

In [ ]:
## Tile names and urls for all tiles apart of the dataset (complete array)

In [ ]:
tile_names_tile_urls_complete_array = fc.add_formatted_and_standard_tile_names_to_tile_names_time_urls(tile_names_tile_urls_complete_array)
print(tile_names_tile_urls_complete_array.shape)

tile_names_tile_urls_complete_array_unique_standard_tile_names, tile_names_tile_urls_complete_array_unique_formatted_tile_names = fc.unique_formatted_standard_tile_names(tile_names_tile_urls_complete_array)
#Save unique standard tile names
np.save("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy", tile_names_tile_urls_complete_array_unique_standard_tile_names)
#pd.DataFrame(np.load("image_download_azure/tile_name_tile_url_complete_array.npy")).to_csv("image_download_azure/tile_name_tile_url_complete_array.csv")

In [ ]:
def get_xml_path_from_root_jpg(df, i):
    """
    """
    file_name = os.path.splitext(df.six_digit_chip_name[i])[0]
    xml_path = os.path.join(df.root[i], "chips_positive_xml", file_name + ".xml") #review how root path was formed 
    tree = et.parse(xml_path)
    root = tree.getroot()
    return(xml_path,root)

def generate_xml(tile_name, tiles_path, tiles_xml_path, tile_bands, tile_height, tile_width):
    """
    """
    #https://www.geeksforgeeks.org/create-xml-documents-using-python/
    #root = et.Element("annotation")
      
    #filename = et.Element("filename")
    #filename.text = tile_name + ".tif" #tilename
    #root.append(filename)
    """
    path = et.Element("path") 
    path.text = tiles_path   #path of database (tiles)
    root.append(path)
    
    source = et.Element("source")
    root.append(source)
    database = et.SubElement(source, "database")
    database.text = "AST Dataset - Complete Tile" #name of database

    

    size = et.Element("size")
    root.append(size)
    width = et.SubElement(size, "width")
    width.text = tile_width #tile width
    height = et.SubElement(size, "height")
    height.text = tile_height #tile height
    depth = et.SubElement(size, "depth")
    depth.text = tile_bands #tile depth
    """   
    #tree = et.ElementTree(root)
    tile_xml_path = os.path.join(tiles_xml_path, tile_name + ".xml")
    print(tile_xml_path)
    #with open (os.path(tiles_xml_complete_dataset_path, tile_name + ".xml"), "wb") as files :
    #    tree.write(files)

In [ ]:
tiles_complete_dataset_path

In [ ]:
unique_positive_jpgs
T

In [ ]:
image_characteristics

In [ ]:
#specify folder that holds tiles in completed dataset

#unique positive jpgs (file names with the file extension)
unique_positive_jpgs = fc.unique_positive_jpgs_from_parent_directory(args.parent_directory)

image_characteristics = fc.image_characteristics(complete_dataset_tiles_folder_path, unique_positive_jpgs)

image_characteristics.to_csv('image_characteristics.csv')
counterin = 0
counternot = 0 

#Check to see how many images are not yet in the image characteristics folder (not verified)
for unique_jpg in unique_positive_jpgs[:,0]:
    if image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counterin += 1
    if not image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counternot += 1
print("images included in the image characteristics csv ",counterin, \
      "images not included in the image characteristics csv \ left to be verified", counternot)

In [ ]:
#for unique_standard_tile_names in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]:
#    if unique
tiles_with_no_images = []
tiles_with_images = []
for standard_tile_name in tqdm_notebook(image_characteristics.standard_tile_name.unique()): #Iterate over all the possible tiles that could be included in the dataset
    print(standard_tile_name)
    #get the image characteristics for the images corresponding to each tile
    images_in_tile = image_characteristics.loc[image_characteristics.standard_tile_name == standard_tile_name] #get the annotated images corresponding to ech time
    indicies = images_in_tile.index
    
    #get the characteristics for each time
    da = xr.open_rasterio(os.path.join(tiles_complete_dataset_path, standard_tile_name +".tif"))
    tile_band, tile_height, tile_width = da.shape[0], da.shape[1], da.shape[2]
    #make the xml for each tile
    generate_xml(standard_tile_name, tiles_complete_dataset_path, tiles_xml_complete_dataset_path, tile_band, tile_height, tile_width)
    #add data from each image xml to the corresponding tile xml
    #for i in indicies:
    #    xml_path, root = get_xml_path_from_root_jpg(images_in_tile, i)
    #if not images_in_tile.empty:

In [ ]:
image_characteristics

In [ ]:
dataset = rasterio.open(os.path.join(tiles_complete_dataset_path, os.listdir(tiles_complete_dataset_path)[0]))
left = dataset.bounds[0]
right = dataset.bounds[2]
bottom = dataset.bounds[1]
top = dataset.bounds[3]
x = dataset.shape[0] 
y = dataset.shape[1] 
np.linspace(left, right, num=x-1)# endpoint=True, retstep=False, dtype=None, axis=0)
# Read the data
## Get tile locations
da = xr.open_rasterio(os.path.join(tiles_complete_dataset_path, os.listdir(tiles_complete_dataset_path)[0]))
# Compute the lon/lat coordinates with rasterio.warp.transform
ny, nx = len(da['y']), len(da['x'])
x, y = np.meshgrid(da['x'], da['y'])
da.crs

## Combine tiles
- For every tile
- identify all annotated images (.xmls) corresponding to the tile
    - identify the position of the image

In [ ]:
#jpg = os.path.splitext(base)[0] #name of tif with the extension removed
#tile_name_formated_name = jpg.rsplit("_",1)[0] #name of tif with the extensio
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
df.loc[df['column_name'].isin(some_values)]

In [ ]:
DomTree = xml.dom.minidom.parse("Catalog.xml")
annotation = DomTree.documentElement

filenamelist = annotation.getElementsByTagName('filename') #[<DOM Element: filename at 0x381f788>]
filename = filenamelist[0]
filename.text()
#objectlist = annotation.getElementsByTagName('object')




def newRunRun(folder):
    xml_files = glob.glob(folder+"/*.xml")
    node = None
    for xmlFile in xml_files:      
        tree = ElementTree.parse(xmlFile)
        root = tree.getroot()
        if node is None:
            node = root
        else:
            elements = root.find("./results")           
            for element in elements._children:
                node[1].append(element)                
    print ElementTree.tostring(node)
    
    
        xml_file = self.chips_xml_list[i]
    # use the parse() function to load and parse an XML file
    tree = et.parse(os.path.join(self.chips_xml_dir, xml_file))
    root = tree.getroot()         

    for obj in root.iter('object'):
        for name in obj.findall('name'):
            if name.text not in correctly_formatted_object:
                name.text = object_dict[name.text]

        if int(obj.find('difficult').text) == 1:
            obj.find('truncated').text = '1'
            obj.find('difficult').text = '0'

folder = "resources"
newRunRun(folder) 

In [ ]:

# Replace <Subscription Key> with your valid subscription key.
subscription_key = "f244aa59ad4f4c05be907b4e78b7c6da"
assert subscription_key

vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v2.0/"

ocr_url = vision_base_url + "ocr"

# Set image_url to the URL of an image that you want to analyze.
image_url = "https://cdn-ayb.akinon.net/cms/2019/04/04/e494dce0-1e80-47eb-96c9-448960a71260.jpg"

headers = {'Ocp-Apim-Subscription-Key': subscription_key}
params  = {'language': 'unk', 'detectOrientation': 'true'}
data    = {'url': image_url}
response = requests.post(ocr_url, headers=headers, params=params, json=data)
response.raise_for_status()

analysis = response.json()

# Extract the word bounding boxes and text.
line_infos = [region["lines"] for region in analysis["regions"]]
word_infos = []
for line in line_infos:
    for word_metadata in line:
        for word_info in word_metadata["words"]:
            word_infos.append(word_info)
word_infos

# Display the image and overlay it with the extracted text.
plt.figure(figsize=(100, 20))
image = Image.open(BytesIO(requests.get(image_url).content))
ax = plt.imshow(image)
texts_boxes = []
texts = []
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0], bbox[1])
    patch  = Rectangle(origin, bbox[2], bbox[3], fill=False, linewidth=3, color='r')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=2, weight="bold", va="top")
#     print(bbox)
    new_box = [bbox[1], bbox[0], bbox[1]+bbox[3], bbox[0]+bbox[2]]
    texts_boxes.append(new_box)
    texts.append(text)
#     print(text)
plt.axis("off")
texts_boxes = np.array(texts_boxes)
texts_boxes

In [ ]:
fc.rename_formatted_chips_images_xmls(complete_dataset_path)